### installing requirements 

In [4]:
!pip install --upgrade pip
!pip install spacy
!pip install transformers
!pip install sentencepiece
!pip install pysimilar
!pip install datasets

     |████████████████████████████████| 2.1 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.2/895.2 KB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 KB 5.7 MB/s eta 0:00:00
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━

### MOUNTING STORAGE TO GOOGLE DRIVE


In [2]:
import gc
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


### LOADING TRANSLATION TRANSFORMERS USING HUGGINGFACE


In [5]:
from transformers import pipeline
to_swahili_translator = pipeline('translation', model='Helsinki-NLP/opus-mt-en-sw')
to_english_translator = pipeline('translation', model='Helsinki-NLP/opus-mt-swc-en')

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/794k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/286M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

### INITIALIZING-SPACY

In [6]:
import spacy

nlp = spacy.load('en_core_web_sm')

### NER TRANSLATION ALGORITHM

In [7]:
from pysimilar import compare

def label_sw(sw_text):
  sw_entities = {}
  en_text = to_english_translator(sw_text)[0]['translation_text']
  sw_text = to_swahili_translator(en_text)[0]['translation_text']
  # en_text = to_english_translator(sw_text)[0]['translation_text']
  # print(en_text)
  if en_text:
    entities = nlp(en_text).ents 
    for entity in entities:
      # print(entity.text, entity.label_)
      back_to_sw = to_swahili_translator(entity.text)
      if not back_to_sw:
        continue 
      back_to_sw = back_to_sw[0]['translation_text']
      # print(back_to_sw)
      # print(compare(back_to_sw, sw_text))
      if back_to_sw in sw_text:
        sw_entities[back_to_sw] = entity.label_
    return [sw_text, sw_entities]

### LOADING THE DATASETS

In [8]:
import pandas as pd
from datasets import load_dataset

# lOADING THE DATASET 
dataset = load_dataset("swahili")

# FORMATING DATASET AS DATAFRAME
train_data = dataset.get('train')
df_train = pd.DataFrame(train_data)
df_train.head()

Generating train split:   0%|          | 0/42069 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3371 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3372 [00:00<?, ? examples/s]

Dataset swahili downloaded and prepared to /root/.cache/huggingface/datasets/swahili/swahili/1.0.0/15bf1d99abb939f83b5da3c798ed55e9803b3ea430f06bf7e003bd073b60172a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

,text
0,taarifa hiyo ilisema kuwa ongezeko la joto la ...
1,aidha ilisema kuwa mwelekeo wa kupungua kwa jo...
2,mwelekeo wa mvua wa septemba hadi desemba ishi...
3,ilifafanua kuwa msimu wa vuli UNK maeneo ambay...
4,katika maeneo hayo mvua zinatarajiwa kunyesha ...


**Function to divide dataset into pieces**

In [ ]:
def divide_by_n(data, pieces=10):
  divisible_pieces = len(data)//pieces
  remainder = len(data)%pieces
  print(divisible_pieces, ',  ', remainder)
  divisible_groups = [data[n*pieces:((n*pieces)+pieces)] for n in range(divisible_pieces)]
  if remainder>0:
    divisible_groups.append(data[-remainder:])
  return divisible_groups

In [ ]:
divide_by_n(list(range(34)), 8)

4 ,   2


[[0, 1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20, 21, 22, 23],
 [24, 25, 26, 27, 28, 29, 30, 31],
 [32, 33]]

### DATASET GENERATION LOGIC

In [10]:
import json

def dump_to_file(anno_data):
  with open('drive/MyDrive/NER-AUTO-LABELER/annotated_data_2.json', 'w') as f:
    json.dump(anno_data, f)

annotated_data = []

def generate_data(train_texts):
    for d_text in train_texts:
      labeled_data = label_sw(d_text)
      annotated_data.append(labeled_data)
      dump_to_file(annotated_data)
      print(labeled_data)

In [11]:
from concurrent.futures import ThreadPoolExecutor 

def main():
  df_train_splitted = (list(df_train['text']), 10)
  with ThreadPoolExecutor(max_workers=10) as process:
    for d_ in df_train_splitted:
      process.submit(generate_data, d_)
    input("Press enter to continue")


In [12]:
main()